Logistic Optimization Analysis with CRIU-AI (LangGraph Implementation)

## Important Note on Tooling Choice

This project was originally intended to be implemented using the CrewAI framework.
However, due to repeated compatibility issues between CrewAI and the Groq LLM
(including OpenAI API key fallback requirements and model resolution errors),
the CrewAI-based execution could not be reliably completed using a Groq API key.

To ensure a fully functional, executable, and conceptually correct CRIU-AI system,
this project is implemented using **LangChain + LangGraph**, which supports Groq
LLMs natively and allows explicit agent-to-agent workflow orchestration.

This implementation:
- Preserves the **Crew-AI philosophy**
- Uses **role-based agents**
- Executes **sequential reasoning**
- Requires **only a Groq API key**
- Avoids OpenAI dependency entirely

The workflow and outputs remain equivalent to a CrewAI-based system and demonstrate
multi-agent collaboration for logistics optimization.


In [3]:
!pip install langchain langgraph langchain-groq pandas


In [4]:
import pandas as pd
from typing import TypedDict, List
from langchain_groq import ChatGroq
from langgraph.graph import StateGraph


In [5]:
# Use your Groq API key here
llm = ChatGroq(
    groq_api_key="",  # replace with actual key
    model_name="llama-3.3-70b-versatile",  # current supported Groq model
    temperature=0.2
)

def llm_invoke(prompt: str):
    return llm.invoke(prompt).content


In [6]:
data = pd.DataFrame({
    "product": ["Laptop", "Mobile", "Headphones"],
    "monthly_demand": [1200, 2500, 1800],
    "current_inventory": [800, 2600, 1000],
    "lead_time_days": [7, 3, 5],
    "transport_cost_per_unit": [12, 8, 5]
})

data


,product,monthly_demand,current_inventory,lead_time_days,transport_cost_per_unit
0,Laptop,1200,800,7,12
1,Mobile,2500,2600,3,8
2,Headphones,1800,1000,5,5


In [7]:
class LogisticsState(TypedDict):
    products: List[dict]
    data_summary: str
    analysis: str
    strategy: str
    final_decision: str


In [8]:
def logistics_analyst(state: LogisticsState) -> LogisticsState:
    prompt = f"""
You are a logistics analyst.

Analyze the following products and logistics data:

{state['products']}

Focus on:
- Demand vs Inventory gaps
- Lead time risks
- Cost inefficiencies

Provide a concise analytical summary.
"""
    response = llm_invoke(prompt)
    return {**state, "analysis": response}


In [9]:
def optimization_strategist(state: LogisticsState) -> LogisticsState:
    prompt = f"""
You are a logistics optimization strategist.

Based on the following analysis:

{state['analysis']}

Create a practical optimization strategy including:
- Inventory replenishment
- Route or cost optimization
- Risk mitigation actions
"""
    response = llm_invoke(prompt)
    return {**state, "strategy": response}


In [10]:
def decision_integrator(state: LogisticsState) -> LogisticsState:
    prompt = f"""
You are a senior logistics decision integrator.

Inputs:
Analysis: {state['analysis']}
Strategy: {state['strategy']}

Task:
Produce a final executive-level decision report including:
1. Key findings
2. Recommendations
3. Expected impact
4. Assumptions and risks
"""
    response = llm_invoke(prompt)
    return {**state, "final_decision": response}


In [11]:
graph = StateGraph(LogisticsState)

graph.add_node("logistics_analyst", logistics_analyst)
graph.add_node("optimization_strategist", optimization_strategist)
graph.add_node("decision_integrator", decision_integrator)

graph.set_entry_point("logistics_analyst")

graph.add_edge("logistics_analyst", "optimization_strategist")
graph.add_edge("optimization_strategist", "decision_integrator")

app = graph.compile()


In [12]:
initial_state = {
    "products": data.to_dict(orient="records"),
    "data_summary": "",
    "analysis": "",
    "strategy": "",
    "final_decision": ""
}


In [13]:
result = app.invoke(initial_state)


In [14]:
print("=== LOGISTICS ANALYSIS ===")
print(result["analysis"])

print("\n=== OPTIMIZATION STRATEGY ===")
print(result["strategy"])

print("\n=== FINAL DECISION (INTEGRATED) ===")
print(result["final_decision"])


=== LOGISTICS ANALYSIS ===
**Logistics Analysis Summary**

Based on the provided data, the following key insights are identified:

1. **Demand vs Inventory gaps**:
	* Laptop: 400 units short (1200 monthly demand - 800 current inventory), indicating a potential stockout risk.
	* Mobile: 100 units excess (2600 current inventory - 2500 monthly demand), suggesting overstocking.
	* Headphones: 800 units short (1800 monthly demand - 1000 current inventory), indicating a potential stockout risk.
2. **Lead time risks**:
	* Laptop: 7-day lead time may exacerbate stockout risks due to high demand and current inventory shortfall.
	* Mobile: 3-day lead time is relatively short, minimizing stockout risks.
	* Headphones: 5-day lead time may contribute to stockout risks due to high demand and current inventory shortfall.
3. **Cost inefficiencies**:
	* Laptop: High transport cost per unit ($12) may be optimized by exploring alternative shipping methods or renegotiating with suppliers.
	* Mobile: Relat

In [15]:
question = "What would happen if the monthly demand for Laptops doubled next month?"

answer = llm_invoke(f"""
You are a logistics expert.

Final report: {result['final_decision']}

Question: {question}

Provide a reasoned response based on the analysis and strategy above.
""")

print("=== AD-HOC QUESTION ANSWER ===")
print(answer)


=== AD-HOC QUESTION ANSWER ===
If the monthly demand for Laptops were to double next month, it would significantly impact our current logistics optimization strategy. Based on the report, we are currently 400 units short of meeting the monthly demand for Laptops. If demand were to double, this shortage would increase to 1,200 units (400 units short of original demand + 800 units of additional demand).

Given this scenario, our replenishment strategy would need to be adjusted to meet the new demand. We would need to expedite replenishment of Laptop inventory to avoid stockout risks and potential lost sales. This could involve:

1. **Emergency replenishment**: We would need to work with our suppliers to expedite shipments and increase the replenishment quantity to meet the new demand.
2. **Air freight**: To mitigate lead time risks, we might consider using air freight instead of the current transportation mode, which would reduce the lead time from 7 days to potentially 1-2 days. However